# Using EAGO's basic optimizer with user-defined subroutines 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

[Robert Gottlieb](https://psor.uconn.edu/person/robert-gottlieb/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

### Overview  
In this section, we construct an optimizer that uses EAGO's basic NLP solution routine with user-defined lower and upper bounding problems. The **EAGO.Optimizer** structure supplies a number of parameters and stored structures that advanced users may find useful for constructing specialized solution routines. For a full review of the EAGO.Optimizer object the reader is directed to the **EAGO.Optimizer** docstring and documentation provided at [https://psorlab.github.io/EAGO.jl/stable/](https://psorlab.github.io/EAGO.jl/stable/).

In this example, we'll forgo extensive integration into the EAGO.Optimizer and simply replace the lower and upper-bounding problems to construct a B&B routine that solves the following problem to global optimality using bounds obtained from interval arithmetic:

$
\begin{align}
&\min_{\mathbf x \in X} \;\; \sin(x_1)x_2^2 - \cos(x_3) / x_4\\
&X = [-10,10]\times[-1,1]\times[-10,10]\times[2,20].
\end{align}
$

We begin importing EAGO, IntervalArithmetic[1], and JuMP[2].

In [29]:
using EAGO, IntervalArithmetic, JuMP

We now define the IntervalExt struct as a subtype of the EAGO.ExtensionType.

In [30]:
struct IntervalExt <: EAGO.ExtensionType end

### Defining a custom lower bounding problem
A valid lower bound is obtained from the lower bound of the natural interval extension using the **IntervalArithmetic.jl** package. The LowerProblem is dispatched using the new **IntervalExt** structure and the **EAGO.GlobalOptimizer** structure, computes the bound using interval arithmetic, and stores the results to the appropriate field of the **EAGO.GlobalOptimizer**. Note that the problem is unconstrained on the domain so we can assume it is always feasible. Further, since the interval bound is constrained along the entire domain associated with a node, no additional cuts will be beneficial and thus we've disabled them using the `_cut_add_flag` field.

In [31]:
import EAGO: lower_problem!
function lower_problem!(t::IntervalExt, x::EAGO.GlobalOptimizer)
    # Retrieve bounds at current node
    n = x._current_node
    lower = n.lower_variable_bounds
    upper = n.upper_variable_bounds
    
    # Define X for the node and compute the interval extension
    x_value = Interval.(lower, upper)
    F = sin(x_value[1])*x_value[2]^2-cos(x_value[3])/x_value[4]
    x._lower_objective_value = F.lo
    x._lower_solution = IntervalArithmetic.mid.(x_value)
    x._lower_feasibility = true
    x._cut_add_flag = false
    
    return
end

lower_problem! (generic function with 3 methods)

### Defining a custom upper bounding problem
Since the problem is unconstained, any feasible point represents a valid upper bound. Thus, if we arbitrarily evaluate the function at the midpoint, we obtain a valid upper bound. This function constructs an upper bound in this manner then stores the results to the appropriate field of the **EAGO.GlobalOptimizer**.

In [32]:
import EAGO.upper_problem!
function EAGO.upper_problem!(t::IntervalExt, x::EAGO.GlobalOptimizer)
    # Retrieve bounds at current node
    n = x._current_node
    lower = n.lower_variable_bounds
    upper = n.upper_variable_bounds
    
    # Compute midpoint value and evaluate at that point
    x_value = 0.5*(upper + lower)
    f_val = sin(x_value[1])*x_value[2]^2-cos(x_value[3])/x_value[4]
    x._upper_objective_value = f_val
    x._upper_solution = x_value
    x._upper_feasibility = true
    
    return
end

### Disable unnecessary routines.
It is entirely possible to disable domain reduction by manipulating keyword arguments supplied to the optimizer. However, for simplicity's sake we'll simply overload the default preprocessing and postprocessing methods and indicate that there are no conditions under which EAGO should cut the node.

In [33]:
import EAGO: preprocess!, postprocess!, cut_condition
function EAGO.preprocess!(t::IntervalExt, x::EAGO.GlobalOptimizer)
    x._preprocess_feasibility = true
    return
end
function EAGO.postprocess!(t::IntervalExt, x::EAGO.GlobalOptimizer)
    x._postprocess_feasibility = true
    return
end
EAGO.cut_condition(t::IntervalExt, x::EAGO.GlobalOptimizer) = false

### Build the JuMP Model and optimize
We now add our optimizer to a JuMP model, provide variable bounds, and optimize.

In [34]:
# Create a factory that specifies the interval extension in EAGO's SubSolver
factory = () -> EAGO.Optimizer(SubSolvers(; t = IntervalExt()))

# Create a JuMP model using the factory, and with the absolute tolerance set by keyword argument
model = Model(optimizer_with_attributes(factory,
                    "absolute_tolerance" => 0.001,
                    "output_iterations" => 200,
                    "unbounded_check" => false,
        ))

# Add variables, bounds, and the objective function
xL = [-10.0, -1.0, -10.0, 2.0]
xU = [10.0, 1.0, 10.0, 20.0]
@variable(model, xL[i] <= x[i=1:4] <= xU[i])
@objective(model, Min, sin(x[1])*x[2]^2 - cos(x[3])/x[4])

# Perform the optimization
JuMP.optimize!(model)

---------------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes     |  Lower Bound  |  Upper Bound  |      Gap      |     Ratio     |     Timer     |   Time Left   |
---------------------------------------------------------------------------------------------------------------------------------
|           200 |            63 |    -1.500E+00 |    -1.310E+00 |     1.902E-01 |     1.268E-01 |          0.00 |       3600.00 |
|           400 |            87 |    -1.500E+00 |    -1.483E+00 |     1.681E-02 |     1.121E-02 |          0.00 |       3600.00 |
|           600 |           137 |    -1.500E+00 |    -1.487E+00 |     1.269E-02 |     8.463E-03 |          0.01 |       3599.99 |
|           734 |           139 |    -1.500E+00 |    -1.499E+00 |     1.039E-03 |     6.926E-04 |          0.01 |       3599.99 |
------------------------------------------------------------------------------------------

### Get information from the JuMP Model object
The objective value, solution, termination status,  and primal status can then be accessed via the standard JuMP interface.

In [35]:
fval = JuMP.objective_value(model)
xsol = JuMP.value.(x)
tstatus = JuMP.termination_status(model)
pstatus = JuMP.primal_status(model)

println("EAGO terminated with a status of $tstatus and a result code of $pstatus.")
println("The optimal value is: $(round(fval,digits=3)), the solution found is $(round.(xsol,digits=4)).")

EAGO terminated with a status of OPTIMAL and a result code of FEASIBLE_POINT.
The optimal value is: -1.499, the solution found is [-1.5698, -0.9998, -0.0024, 2.0022].


### Advice for more advanced constructions
The default `lower_problem!` and `upper_problem!` should be used templates for error handling and retreiving information from MOI models. 

Essentially all of EAGO's subroutines stored to a field in the `EAGO.Optimizer` structure can be reset as user-defined functions.

### References
1. IntervalArithmetic.jl [Computer software] (2019). Retrieved from https://github.com/JuliaIntervals/IntervalArithmetic.jl
2. Iain Dunning and Joey Huchette and Miles Lubin. JuMP: A Modeling Language for Mathematical Optimization, *SIAM Review*, **SIAM** 59 (2017), pp. 295-320.